In [2]:
import pandas as pd
from core_functions import *
import os

Note before starting:
    <p>All data files must be in data/ folder<p>
    <p>All results will be generated in results/ folder including an image (in PNG) and an Excel file<p>
    <p> All file names should be written between 

In [17]:
#File path of the functional database (it need to be in gmt format!)
#It should be in data/MSigDB/ file
db_name = '../data/MSigDB/go_mf.gmt'

#Filepath of the gene sets of interest to perform enrichment
filename = '../data/example_groups_file.xls'

#File path of the list of functional terms you do not want to display, by default none. 
filenameGO = 'No_file'

#Minimum of genes intersection your gene set and each pathway to consider the enrichment
#by default set to 0
mngene = 0

#File path of the output excel file where you want your results. 
out_ex = '../results/example_output.xls'

#Corrected P-value significance threshold
fs = 0.05

#Generate results folder if it is not already there
try:
    os.stat('../results/')
except:
    os.mkdir('../results/')       

## Functional enrichment

In [1]:
if filename == 'No_file':
    print('PLEASE SPECIFY A GROUP FILE. It should be in data/ folder')
else:
    #Load data
    clusters = pd.read_excel('../data/'+filename)
    cluster_genes = {}
    for cluster in clusters.columns.tolist():
        cluster_genes[cluster] = clusters[cluster].dropna().tolist()
    
    #Load functional annotation data
    genesets = {}
    allgenes = []
    ffile = open(db_name,'r')
    for line in ffile.readlines():
        f = line.strip().split('\t')
        genesets[f[0]] = f[2:]
        allgenes += f[2:]
    ffile.close()
    allgenes = set(allgenes)
    
    #Functional enrichment
    allenrichment_results = enrichment_all_groups(genesets,mngene,filenameGO,allgenes,cluster_genes,float(fs))
    
    if len(allenrichment_results) > 0:
        #Plot heatmap
        plot_heatmap(allenrichment_results,clusters,cluster_genes,float(fs))

        #Generate tabe with all results
        allenrichment_results[['GROUP','TERM','TERM_GENE_SET_SIZE','PVALUE','QVALUE','ENRICHED_GENES']].to_excel(out_ex,index=False)
    else:
        print('Any gene set from',dbn.value,'was enriched by',filename)

NameError: name 'filename' is not defined

Explanation of the excel file generated
<li>GROUP: name of the user gene set of interest</li>
<li>TERM: pathway enriched</li>
<li>TERM_GENE_SET_SIZE: number of genes in pathway enriched</li>
<li>PVALUE: P-value of the enrichment</li>
<li>QVALUE: Corrected P-values (FDR bh) of the enrichment (the one to look at!)</li>
<li>ENRICHED_GENES: list of genes in the pathway and in the user gene set</li>